In [24]:
# pip install pyspark
!pip install elasticsearch

In [25]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.elasticsearch:elasticsearch-spark-30_2.12:7.16.2,com.fasterxml.jackson.module:jackson-module-scala_2.12:2.13.0 pyspark-shell'

In [26]:
from pyspark.sql import SparkSession

# Create a SparkSession object
spark = SparkSession.builder.appName("ElasticsearchIndex").getOrCreate()

In [27]:
# read the CSV file into a DataFrame
FILE_INPUT='./ml/trans.csv'
print("Load  files")
%time transactions_df = spark.read.format('csv').option('header',True).option('inferSchema',True).load(FILE_INPUT)

Load  files
CPU times: user 5.3 ms, sys: 2.14 ms, total: 7.44 ms
Wall time: 1.74 s


In [28]:
# Define the index mapping
index_mapping = {
    "mappings": {
        "properties": {
            "TRANSACTION_ID": {"type": "keyword"},
            "TX_DATETIME": {"type": "date", "format": "yyyy-MM-dd HH:mm:ss"},
            "CUSTOMER_ID": {"type": "keyword"},
            "TERMINAL_ID": {"type": "keyword"},
            "TX_AMOUNT": {"type": "double"},
            "TX_TIME_SECONDS": {"type": "integer"},
            "TX_TIME_DAYS": {"type": "integer"},
            "TX_FRAUD": {"type": "integer"},
            "TX_FRAUD_SCENARIO": {"type": "keyword"},
            "TX_DURING_WEEKEND": {"type": "integer"},
            "TX_DURING_NIGHT": {"type": "integer"},
            "CUSTOMER_ID_NB_TX_1DAY_WINDOW": {"type": "integer"},
            "CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW": {"type": "double"},
            "CUSTOMER_ID_NB_TX_7DAY_WINDOW": {"type": "integer"},
            "CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW": {"type": "double"},
            "CUSTOMER_ID_NB_TX_30DAY_WINDOW": {"type": "integer"},
            "CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW": {"type": "double"},
            "TERMINAL_ID_NB_TX_1DAY_WINDOW": {"type": "integer"},
            "TERMINAL_ID_RISK_1DAY_WINDOW": {"type": "double"},
            "TERMINAL_ID_NB_TX_7DAY_WINDOW": {"type": "integer"},
            "TERMINAL_ID_RISK_7DAY_WINDOW": {"type": "double"},
            "TERMINAL_ID_NB_TX_30DAY_WINDOW": {"type": "integer"},
            "TERMINAL_ID_RISK_30DAY_WINDOW": {"type": "double"}
        }
    }
}

In [29]:
from elasticsearch import Elasticsearch
from elasticsearch.exceptions import RequestError

# Create an Elasticsearch client
es = Elasticsearch([{'host': 'elasticsearch', 'port': 9200, 'scheme': 'http'}])

# Define the Elasticsearch index name //---and document type
es_index = "my_transactions_index0"

# Create a mapping for the Elasticsearch index

# Check if the index exists, and create it if it does not
# Create the Elasticsearch index if it doesn't exist
if not es.indices.exists(index=es_index):
    try:
        es.indices.create(index=es_index, body=index_mapping)
    except RequestError as e:
        print(f"Index creation failed: {e}")
        exit(1)
    print(f"Created Elasticsearch index '{es_index}'")


/tmp/ipykernel_18046/2130625194.py:14: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  if not es.indices.exists(index=es_index):


In [30]:
query = transactions_df.write \
    .format("org.elasticsearch.spark.sql") \
    .option("es.nodes", "localhost") \
    .option("es.port", "9200") \
    .option("es.resource", es_index) \
    .option("es.mapping.id", "TRANSACTION_ID").save()
query.awaitTermination()

Py4JJavaError: An error occurred while calling o90.save.
: org.elasticsearch.hadoop.EsHadoopIllegalArgumentException: Cannot detect ES version - typically this happens if the network/Elasticsearch cluster is not accessible or when targeting a WAN/Cloud instance without the proper setting 'es.nodes.wan.only'
	at org.elasticsearch.hadoop.rest.InitializationUtils.discoverClusterInfo(InitializationUtils.java:403)
	at org.elasticsearch.spark.sql.ElasticsearchRelation.cfg$lzycompute(DefaultSource.scala:225)
	at org.elasticsearch.spark.sql.ElasticsearchRelation.cfg(DefaultSource.scala:223)
	at org.elasticsearch.spark.sql.ElasticsearchRelation.isEmpty(DefaultSource.scala:624)
	at org.elasticsearch.spark.sql.DefaultSource.createRelation(DefaultSource.scala:110)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:47)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:75)
	at org.apache